In [47]:
import pandas as pd
import os
import numpy as np

In [48]:
def sord(df, code_len):
    new_df = pd.DataFrame(columns=['time', 'code', 'IM_PQ', 'IM_Q', 'EX_PQ', 'EX_Q'])

    pd.DataFrame(columns=['time', 'code', 'IM_PQ', 'IM_Q', 'EX_PQ', 'EX_Q'])
    
    # 处理进口金额
    import_amount = df[df['指标'] == '进口金额（美元）']
    import_amount = import_amount[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'IM_PQ'})
    
    # 处理进口量（仅第一数量）
    import_quantity = df[df['指标'] == '进口数量（第一数量）']
    import_quantity = import_quantity[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'IM_Q'})
    
    # 处理出口金额
    export_amount = df[df['指标'] == '出口金额（美元）']
    export_amount = export_amount[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'EX_PQ'})
    
    # 处理出口量（仅第一数量）
    export_quantity = df[df['指标'] == '出口数量（第一数量）']
    export_quantity = export_quantity[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'EX_Q'})
    
    # 合并数据
    new_df = (
        import_amount
        .merge(export_amount, on=['time', 'code'], how='outer')
        .merge(import_quantity, on=['time', 'code'], how='outer')
        .merge(export_quantity, on=['time', 'code'], how='outer')
    )
    
    # 仅保留code列的前四位字符
    new_df['code'] = new_df['code'].astype(str).str[:code_len]
    
    # 填充缺失值为 "空"
    new_df = new_df.fillna(pd.NA)
    new_df.head()
    return new_df


In [49]:
so1 = "../data/4pos"
so2 = "../data/6pos"

In [50]:
def merge_csv_files(folder_path, code_len):
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    df_list = [sord(pd.read_csv(file, encoding='gbk')[:-2], code_len) for file in all_files]
    return pd.concat(df_list, ignore_index=True)


In [51]:
# 先搞一下第一个的
so1_sorted = merge_csv_files(so1, 4)

iv = pd.read_csv("../data/iv.list.csv")
rate = pd.read_csv('../data/rate.csv', encoding='utf-8')

In [52]:
def calculate_prices(df):
    """
    计算进口和出口的单价，避免除以零或 NaN 的情况。
    """
    import numpy as np
    
    df["IM_P"] = df.apply(
        lambda row: row["IM_PQ"] / row["IM_Q"] if row["IM_Q"] not in [0, np.nan] else np.nan, axis=1
    )
    
    df["EX_P"] = df.apply(
        lambda row: row["EX_PQ"] / row["EX_Q"] if row["EX_Q"] not in [0, np.nan] else np.nan, axis=1
    )
    
    return df


In [53]:
def extract_year(df):
    """
    从 time 列提取年份并创建 year 列。
    """
    df["year"] = df["time"].str[-4:].astype(int)
    return df


In [54]:
def merge_with_iv(df, iv):
    """
    将数据与 iv 数据集按照年份进行合并。
    """
    df['year'] = df['year'].astype(int)
    iv['year'] = iv['year'].astype(int)
    return df.merge(iv, on='year', how='left')


In [55]:
def calculate_logs(df):
    """
    计算 GDP、人口、FDI、进口单价、出口单价、进口数量、出口数量的对数。
    """
    df["lgdp"] = df["gdp"].apply(np.log)
    df["lpop"] = df["pop"].apply(np.log)
    df["lfdi"] = df["fdi"].apply(np.log)
    df["lexp"] = df['EX_P'].apply(np.log)
    df["limp"] = df['IM_P'].apply(np.log)
    df["lexq"] = df['EX_Q'].apply(np.log)
    df["limq"] = df['IM_Q'].apply(np.log)

    return df


In [56]:
def add_metadata(df):
    """
    添加 code 长度信息，并计算缺失情况。
    """
    df['len'] = df['code'].apply(len)
    
    df['dropped'] = (
        (df["IM_PQ"].isna()) & 
        (df["EX_PQ"].isna()) & 
        (df["IM_Q"].isna()) & 
        (df["EX_Q"].isna())
    ).astype(int)

    df['missing1'] = (
        (df["IM_PQ"].isna()) |
        (df["EX_PQ"].isna()) |
        (df["IM_Q"].isna()) | 
        (df["EX_Q"].isna())
    ).astype(int)

    return df


In [57]:
def add_rate(df):
    df = df.merge(rate, on='time', how='left')
    return df


In [58]:
# 计算进口和出口单价
so1_sorted = calculate_prices(so1_sorted)

# 提取年份信息
so1_sorted = extract_year(so1_sorted)

# 合并 iv 数据
so1_sorted = merge_with_iv(so1_sorted, iv)

# 添加汇率
so1_sorted = add_rate(so1_sorted)

# 计算对数变量
so1_sorted = calculate_logs(so1_sorted)

# 添加元数据
so1_sorted = add_metadata(so1_sorted)



In [59]:
so1_sorted

,time,code,IM_PQ,EX_PQ,IM_Q,EX_Q,IM_P,EX_P,year,fdi,...,lgdp,lpop,lfdi,lexp,limp,lexq,limq,len,dropped,missing1
0,01-2017,0703,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,13.631631,11.849476,16.427931,NaN,NaN,NaN,NaN,4,1,1
1,01-2017,0704,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,13.631631,11.849476,16.427931,NaN,NaN,NaN,NaN,4,1,1
2,01-2017,0706,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,13.631631,11.849476,16.427931,NaN,NaN,NaN,NaN,4,1,1
3,01-2017,0709,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,13.631631,11.849476,16.427931,NaN,NaN,NaN,NaN,4,1,1
4,01-2017,0710,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,13.631631,11.849476,16.427931,NaN,NaN,NaN,NaN,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5380,12-2023,0508,NaN,NaN,NaN,NaN,NaN,NaN,2023,16325000.0,...,14.047084,11.856281,16.608208,NaN,NaN,NaN,NaN,4,1,1
5381,12-2023,0511,NaN,NaN,NaN,NaN,NaN,NaN,2023,16325000.0,...,14.047084,11.856281,16.608208,NaN,NaN,NaN,NaN,4,1,1
5382,12-2023,0602,NaN,NaN,NaN,NaN,NaN,NaN,2023,16325000.0,...,14.047084,11.856281,16.608208,NaN,NaN,NaN,NaN,4,1,1
5383,12-2023,0603,NaN,NaN,NaN,NaN,NaN,NaN,2023,16325000.0,...,14.047084,11.856281,16.608208,NaN,NaN,NaN,NaN,4,1,1


In [61]:
so1_sorted.isin([-np.inf, np.inf]).sum()


time         0
code         0
IM_PQ        0
EX_PQ        0
IM_Q         0
EX_Q         0
IM_P         0
EX_P         0
year         0
fdi          0
gdp          0
pop          0
tea          0
btea         0
gtea         0
rate         0
lgdp         0
lpop         0
lfdi         0
lexp         0
limp         0
lexq         0
limq        18
len          0
dropped      0
missing1     0
dtype: int64

In [63]:
def save(df, save_path):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.to_stata(save_path, write_index=False)

In [64]:
save(so1_sorted, "../data/dt/4.dta")